In [1]:
from configparser import ConfigParser
import psycopg2
import requests
from typing import Dict, List

In [2]:
def load_connection_info(
    ini_filename: str
) -> Dict[str, str]:
    parser = ConfigParser()
    parser.read(ini_filename)
# Create a dictionary of the variables stored under the "postgresql" section of the .ini
    conn_info = {param[0]: param[1] for param in
parser.items("postgresql")}
    return conn_info



In [3]:
def create_db(
    conn_info: Dict[str, str],
) -> None:
    psql_connection_string = "user="\
    f"{conn_info['user']} password={conn_info['password']}"
    conn = psycopg2.connect(psql_connection_string)
    cur = conn.cursor()
    conn.autocommit = True
    sql_query = f"CREATE DATABASE {conn_info['database']}"
    try:
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        cur.close()
    else:
            conn.autocommit = False

In [26]:
def create_table(
    sql_query: str,
    conn: psycopg2.extensions.connection,
    cur: psycopg2.extensions.cursor
) -> None:
    try:
 # Execute the table creation query
        cur.execute(sql_query)
    except Exception as e:
        print(f"{type(e).__name__}: {e}")
        print(f"Query: {cur.query}")
        conn.rollback()
        cur.close()
    else:
        conn.commit()

In [4]:
conn_info = load_connection_info("db.ini")
create_db(conn_info)




DuplicateDatabase: ОШИБКА:  база данных "weather" уже существует

Query: b'CREATE DATABASE weather'


In [5]:
connection = psycopg2.connect(**conn_info)
cursor = connection.cursor()

In [6]:
location_sql = """
    CREATE TABLE IF NOT EXISTS location (
         id SERIAL PRIMARY KEY,
         name VARCHAR(200) UNIQUE NOT NULL,
         region VARCHAR(200) NOT NULL,
         country VARCHAR(200) NOT NULL,
         tz_id VARCHAR(200) NOT NULL,
         local_time TIMESTAMP NOT NULL,
         last_update TIMESTAMP NOT NULL DEFAULT now(),
         local_time_epoch NUMERIC(10) NOT NULL,
         lat NUMERIC(5, 2) NOT NULL,
         lon NUMERIC(5, 2) NOT NULL
         );
"""
create_table(location_sql, connection, cursor)

NameError: name 'create_table' is not defined

In [46]:
# Create the "condition" tablet
condition_sql = """
    CREATE TABLE IF NOT EXISTS condition (
         id SERIAL PRIMARY KEY,
         icon VARCHAR(200) NOT NULL,
         text VARCHAR(50) NOT NULL,
         last_update TIMESTAMP NOT NULL DEFAULT now(),
         code NUMERIC(10) NOT NULL
         );
"""
create_table(condition_sql, connection, cursor)

InterfaceError: cursor already closed
Query: b'\n    CREATE TABLE IF NOT EXISTS current (\n         id SERIAL PRIMARY KEY,\n         condition_id SERIAL REFERENCES condition(id) MATCH SIMPLE\nON UPDATE CASCADE ON DELETE RESTRICT,\n         wind_dir VARCHAR(3) NOT NULL,\n         last_update TIMESTAMP NOT NULL DEFAULT now(),\n         data_published_epoch NUMERIC(10) NOT NULL,\n         data_published TIMESTAMP NOT NULL,\n         temp_c NUMERIC(10,2) NOT NULL,\n         temp_f NUMERIC(10,2) NOT NULL,\n         wind_mph NUMERIC(10,2) NOT NULL,\n         wind_kph NUMERIC(10,2) NOT NULL,\n         wind_degree NUMERIC(10) NOT NULL,\n         pressure_mb NUMERIC(10, 2) NOT NULL,\n         pressure_in NUMERIC(10, 2) NOT NULL,\n         precip_mm NUMERIC(10, 2) NOT NULL,\n         precip_in NUMERIC(10, 2) NOT NULL,\n        69\n         humidity NUMERIC(10, 2) NOT NULL,\n         feelslike_c NUMERIC(10, 2) NOT NULL,\n         feelslike_f NUMERIC(10, 2) NOT NULL,\n         vis_km NUMERIC(10, 2

In [7]:
current_sql = """
    CREATE TABLE IF NOT EXISTS current (
        id SERIAL PRIMARY KEY,
        condition_id SERIAL REFERENCES condition(id) MATCH SIMPLE
ON UPDATE CASCADE ON DELETE RESTRICT,
    wind_dir VARCHAR(3) NOT NULL,
    last_update TIMESTAMP NOT NULL DEFAULT now(),
    data_published_epoch NUMERIC(10) NOT NULL,
    data_published TIMESTAMP NOT NULL,
    temp_c NUMERIC(10,2) NOT NULL,
    temp_f NUMERIC(10,2) NOT NULL,
    wind_mph NUMERIC(10,2) NOT NULL,
    wind_kph NUMERIC(10,2) NOT NULL,
    wind_degree NUMERIC(10) NOT NULL,
    pressure_mb NUMERIC(10, 2) NOT NULL,
    pressure_in NUMERIC(10, 2) NOT NULL,
    precip_mm NUMERIC(10, 2) NOT NULL,
    precip_in NUMERIC(10, 2) NOT NULL,
    humidity NUMERIC(10, 2) NOT NULL,
    feelslike_c NUMERIC(10, 2) NOT NULL,
    feelslike_f NUMERIC(10, 2) NOT NULL,
    vis_km NUMERIC(10, 2) NOT NULL,
    vis_miles NUMERIC(10, 2) NOT NULL,
    uv NUMERIC(10, 2) NOT NULL,
    gust_mph NUMERIC(10, 2) NOT NULL,
    gust_kph NUMERIC(10, 2) NOT NULL,
    is_day BOOLEAN NOT NULL,
    cloud BOOLEAN NOT NULL
 );

"""
create_table(current_sql, connection, cursor)

NameError: name 'create_table' is not defined

In [64]:
# Create the "weather" table
weather_sql = """
    CREATE TABLE IF NOT EXISTS weather (
        id SERIAL PRIMARY KEY,
        location_id SERIAL REFERENCES location(id) MATCH SIMPLE ON
UPDATE CASCADE ON DELETE RESTRICT,
        current_id SERIAL REFERENCES location(id) MATCH SIMPLE ON
UPDATE CASCADE ON DELETE RESTRICT,
        last_update TIMESTAMP NOT NULL DEFAULT now()
 );
"""
create_table(weather_sql, connection, cursor)


InterfaceError: cursor already closed
Query: b'\n    CREATE TABLE IF NOT EXISTS weather (\n        id SERIAL PRIMARY KEY,\n        location_id SERIAL REFERENCES location(id) MATCH SIMPLE ON\nUPDATE CASCADE ON DELETE RESTRICT,\n        current_id SERIAL REFERENCES location(id) MATCH SIMPLE ON\nUPDATE CASCADE ON DELETE RESTRICT,\n        last_update TIMESTAMP NOT NULL DEFAULT now()\n );\n'


InterfaceError: connection already closed

In [8]:
connection.close()
cursor.close()


In [9]:
from threading import Timer

In [10]:
def weather_callback():
    payload = {"key":"38b986140543415595f134926231710",
                "q":"Paris"}
    r = requests.get(
         "http://api.weatherapi.com/v1/current.json",
         params=payload)
    data = r.json()
    print("Temperature in Paris is"
          f"{data['current']['temp_c']} C"
          f" / {data['current']['temp_f']} F")
    return data

In [11]:
t = Timer(interval=30, function=weather_callback)
t.daemon = True

In [ ]:
import time
try:
    t = Timer(interval=30, function=weather_callback)
    t.daemon = True
    t.start()
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    t.cancel()

In [ ]:
[weatherapi]
key=38b986140543415595f134926231710
q=Paris
